<a href="https://colab.research.google.com/github/prashantkodali/hashtagseg/blob/master/dataset_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading and Loading data

In [95]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install datasets transformers seqeval conllu wandb

In [49]:
from tqdm import tqdm
from conllu.models import TokenList, Token

# conllpp

In [40]:
file = open("/content/gdrive/MyDrive/hashtagseg/conllpp_train.txt","r+") 

In [41]:
!wc -l /content/gdrive/MyDrive/hashtagseg/conllpp_train.txt

219553 /content/gdrive/MyDrive/hashtagseg/conllpp_train.txt


In [42]:
ner = []
separated = []
lines = file.readlines()

In [43]:
len(lines)

219553

In [44]:
sentence = ''
labels = ''
sentence_sep = ''
count = 0
x_fin = 219553
x = 0
for i,line in enumerate(lines):

    if i< 10:
      print(line.split())
      print(line.lower().split())

    x += 1
    if x == x_fin:
        break
    if line == '-DOCSTART- -X- -X- O\n':
        continue
    if line != '\n':
        word = line.split()[0]
        sentence += word
        sentence_sep += '_' + word
        label = line.split()[-1].strip('\n').split('-')[0]
        if label == 'B':
            labels += 'B'
            for i in range(len(word)-1):
                labels += 'B'
        elif label == 'I':
            for i in range(len(word)):
                labels += 'I'
        elif label == 'O':
            for i in range(len(word)):
                labels += 'O'
        count += 1
    else:
        if count != 0:
            ner.append((sentence.lower(), labels, count))
            # separated.append((sentence_sep[1:], labels))
            sentence = ''
            labels = ''
            count = 0

['-DOCSTART-', '-X-', '-X-', 'O']
['-docstart-', '-x-', '-x-', 'o']
[]
[]
['EU', 'NNP', 'B-NP', 'B-ORG']
['eu', 'nnp', 'b-np', 'b-org']
['rejects', 'VBZ', 'B-VP', 'O']
['rejects', 'vbz', 'b-vp', 'o']
['German', 'JJ', 'B-NP', 'B-MISC']
['german', 'jj', 'b-np', 'b-misc']
['call', 'NN', 'I-NP', 'O']
['call', 'nn', 'i-np', 'o']
['to', 'TO', 'B-VP', 'O']
['to', 'to', 'b-vp', 'o']
['boycott', 'VB', 'I-VP', 'O']
['boycott', 'vb', 'i-vp', 'o']
['British', 'JJ', 'B-NP', 'B-MISC']
['british', 'jj', 'b-np', 'b-misc']
['lamb', 'NN', 'I-NP', 'O']
['lamb', 'nn', 'i-np', 'o']


In [45]:
len(ner)

14040

In [46]:
ner[0]

('eurejectsgermancalltoboycottbritishlamb.',
 'BBOOOOOOOBBBBBBOOOOOOOOOOOOOBBBBBBBOOOOO',
 9)

In [50]:
allsamples = []

for sample in tqdm(ner):

  assert len(list(sample[0])) == len(list(sample[1]))

  tl = TokenList()

  for ind,(c,a) in enumerate(zip (list(sample[0].lower()),list(sample[1]))):
    # print(ind,c,a)
    tl.append(Token(id =ind,form=c,tag= a))

  # print(tl)

  allsamples.append(tl)

#     sentence2 = TokenList([
# ...    Token(id=(1, "-", 2), form="It's"),
# ...    Token(id=1, form="It"),
# ...    Token(id=2, form="is"),
# ... ])
    # print(c,'\t',a)
  # break

100%|██████████| 14040/14040 [00:02<00:00, 6950.93it/s]


In [51]:
with open('test.conll', 'w') as f:
  f.writelines([sentence.serialize() + "\n" for sentence in allsamples])

# BTC

In [65]:
file = open("/content/gdrive/MyDrive/hashtagseg/BTC.txt","r") 

In [66]:
!wc -l /content/gdrive/MyDrive/hashtagseg/BTC.txt

17612 /content/gdrive/MyDrive/hashtagseg/BTC.txt


In [67]:
ner = []
separated = []
lines = file.readlines()

In [68]:
len(lines)

17612

In [69]:
sentence = ''
labels = ''
sentence_sep = ''
count = 0
x_fin = 17612
x = 0
for i,line in enumerate(lines):

    if i< 10:
      print(line.split())
      print(line.lower().split())

    x += 1
    if x == x_fin:
        break
    if line != '\n':
        word = line.split()[0]
        sentence += word
        sentence_sep += '_' + word
        label = line.split()[-1].strip('\n').split('-')[0]
        if label == 'B':
            labels += 'B'
            for i in range(len(word)-1):
                labels += 'B'
        elif label == 'I':
            for i in range(len(word)):
                labels += 'I'
        elif label == 'O':
            for i in range(len(word)):
                labels += 'O'
        count += 1
    else:
        if count != 0:
            ner.append((sentence.lower(), labels, count))
            # separated.append((sentence_sep[1:], labels))
            sentence = ''
            labels = ''
            count = 0

['I', 'O']
['i', 'o']
['hate', 'O']
['hate', 'o']
['the', 'O']
['the', 'o']
['words', 'O']
['words', 'o']
['chunder', 'O']
['chunder', 'o']
[',', 'O']
[',', 'o']
['vomit', 'O']
['vomit', 'o']
['and', 'O']
['and', 'o']
['puke', 'O']
['puke', 'o']
['.', 'O']
['.', 'o']


In [79]:
len(ner)

999

In [80]:
len(list(ner[0]))

3

In [81]:
len(list(ner[1]))

3

In [88]:
allsamples = []

for sample in tqdm(ner):

  try:
    assert len(list(sample[0])) == len(list(sample[1]))
  except AssertionError:
    print(sample[0])

  tl = TokenList()

  for ind,(c,a) in enumerate(zip (list(sample[0].lower()),list(sample[1]))):
    # print(ind,c,a)
    tl.append(Token(id =ind,form=c,tag= a))

  # print(tl)

  allsamples.append(tl)

#     sentence2 = TokenList([
# ...    Token(id=(1, "-", 2), form="It's"),
# ...    Token(id=1, form="It"),
# ...    Token(id=2, form="is"),
# ... ])
    # print(c,'\t',a)
  # break

100%|██████████| 999/999 [00:00<00:00, 5873.42it/s]

amazonnedenbukadarbaşarılı?i̇ştebuyüzdenhttp://t.co/9lg4yw93


In [89]:
len(allsamples)

999

In [90]:
with open('test.conll', 'a') as f:
  f.writelines([sentence.serialize() + "\n" for sentence in allsamples])

# Hindi

In [96]:
file = open("/content/gdrive/MyDrive/hashtagseg/hindi.txt","r") 

In [97]:
!wc -l /content/gdrive/MyDrive/hashtagseg/hindi.txt

60874 /content/gdrive/MyDrive/hashtagseg/hindi.txt


In [98]:
ner = []
separated = []
lines = file.readlines()

In [99]:
len(lines)

60874

In [9]:
!pip install indic-transliteration

In [10]:
from indictrans import Transliterator

ModuleNotFoundError: ignored

In [ ]:
sentence = ''
labels = ''
sentence_sep = ''
count = 0
x_fin = 17612
x = 0
trn = Transliterator(source='hin', target='eng', build_lookup=True)
for i,line in enumerate(lines):

    if i< 10:
      print(line.split())
      print(line.lower().split())

    x += 1
    if x == x_fin:
        break
    if line != '\n':
        word = line.split()[0].split(':')[1]
        word = trn.transform(word)

        sentence += word
        sentence_sep += '_' + word
        label = line.split()[-1].strip('\n').split('-')[0]
        
        if label == 'B':
            labels += 'B'
            for i in range(len(word)-1):
                labels += 'B'
        elif label == 'I':
            for i in range(len(word)):
                labels += 'I'
        elif label == 'O':
            for i in range(len(word)):
                labels += 'O'
        count += 1
    else:
        if count != 0:
            ner.append((sentence.lower(), labels, count))
            # separated.append((sentence_sep[1:], labels))
            sentence = ''
            labels = ''
            count = 0

['I', 'O']
['i', 'o']
['hate', 'O']
['hate', 'o']
['the', 'O']
['the', 'o']
['words', 'O']
['words', 'o']
['chunder', 'O']
['chunder', 'o']
[',', 'O']
[',', 'o']
['vomit', 'O']
['vomit', 'o']
['and', 'O']
['and', 'o']
['puke', 'O']
['puke', 'o']
['.', 'O']
['.', 'o']


In [ ]:
len(ner)

999

In [ ]:
len(list(ner[0]))

3

In [ ]:
len(list(ner[1]))

3

In [ ]:
allsamples = []

for sample in tqdm(ner):

  try:
    assert len(list(sample[0])) == len(list(sample[1]))
  except AssertionError:
    print(sample[0])

  tl = TokenList()

  for ind,(c,a) in enumerate(zip (list(sample[0].lower()),list(sample[1]))):
    # print(ind,c,a)
    tl.append(Token(id =ind,form=c,tag= a))

  # print(tl)

  allsamples.append(tl)

#     sentence2 = TokenList([
# ...    Token(id=(1, "-", 2), form="It's"),
# ...    Token(id=1, form="It"),
# ...    Token(id=2, form="is"),
# ... ])
    # print(c,'\t',a)
  # break

100%|██████████| 999/999 [00:00<00:00, 5873.42it/s]

amazonnedenbukadarbaşarılı?i̇ştebuyüzdenhttp://t.co/9lg4yw93


In [ ]:
len(allsamples)

999

In [ ]:
with open('test.conll', 'a') as f:
  f.writelines([sentence.serialize() + "\n" for sentence in allsamples])

# do not execute - proto

In [ ]:
# def preproc(hashtag,annotation):

#   tokenized_tokens = tokenizer.convert_ids_to_tokens(tokenizer(hashtag)['input_ids'])

#   print(tokenized_tokens[1:-1])

#   splits = [len(token[2:]) if token[:2]=='##' else len(token) for token in tokenized_tokens[1:-1]]
  
#   print(splits)

#   splits = np.cumsum(splits).tolist()

#   print(splits)

#   splits = [0] + splits[:-1]

#   print(splits)

#   for i,j in zip(splits,splits[1:]+[None]):
#     print(i,j)
#   parts = [annotation[i:j] for i,j in zip(splits, splits[1:]+[None])]


#   print(parts)

#   # [annotation for ind,val in zip(splits,splits[1:])]


#   # [if token[0] == '#' for token in tokenized_tokens]


In [ ]:
# def preproc2(hashtag,annotations):
#   tokenized_input = tokenizer(list(hashtag), is_split_into_words=True)
#   print(len(tokenized_input['input_ids']),len(annotations))


In [ ]:
# hashtag = "EUrejectsGermancalltoboycottBritishlamb."
# annotations = "BBOOOOOOOBBBBBBOOOOOOOOOOOOOBBBBBBBOOOOO"

# preproc2(hashtag,annotations)

# # tokenized_input = tokenizer(hashtag, is_split_into_words=False)
# # tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
# # print(tokens)

In [ ]:
# annotations[37:None]
# hashtag[37:None]

# Preprocessing, tokenisation

In [11]:
model_checkpoint = "bert-base-multilingual-uncased"
batch_size = 8

from datasets import load_dataset, load_metric
import numpy as np

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
from datasets import load_dataset, load_metric

In [14]:
datasets_ner = load_dataset('/content/gdrive/MyDrive/hashtagseg/loadDataset.py',)

['B', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'B', 'B', 'B', 'B', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'O', 'O', 'O', 'O', 'O']
Dataset universal_dependencies downloaded and prepared to /root/.cache/huggingface/datasets/universal_dependencies/default/2.7.0/2eac4dd0af5182f0eac3952937c72d8637086dbf0c59e22126f5c6e063195943. Subsequent calls will reuse this data.


In [15]:
import datasets
datasets_ner = datasets.Dataset.train_test_split(datasets_ner['train'],test_size=0.20,shuffle=True,seed=1234)


In [16]:
datasets_ner

DatasetDict({
    train: Dataset({
        features: ['idx', 'tags', 'tokens'],
        num_rows: 14088
    })
    test: Dataset({
        features: ['idx', 'tags', 'tokens'],
        num_rows: 3522
    })
})

In [17]:
datasets_ner['train'][0]

{'idx': '3569',
 'tags': [0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 'tokens': 'c r o f t 2 9 - 6 - 6 4 - 1 , c o r k 1 4 . 3 - 4 - 4 5 - 1 , s a l i s b u r y 1 7 - 0 - 9 1 - 0'}

In [18]:
assert len(datasets_ner['train'][0]['tags']) == len(datasets_ner['train'][0]['tokens'].split())
# len(datasets_ner['train'][0]['tags']), len(datasets_ner['train'][0]['text'].split())


In [19]:
datasets_ner["train"].features[f"tags"]

Sequence(feature=ClassLabel(num_classes=3, names=['B', 'I', 'O'], names_file=None, id=None), length=-1, id=None)

In [20]:
label_list = datasets_ner["train"].features[f"tags"].feature.names
label_list

['B', 'I', 'O']

In [21]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
label_all_tokens = True

In [23]:
def tokenize_and_align_labels(examples):

    # print(type(examples), examples['tokens'])

    for ind,tl in enumerate(examples['tokens']):
      examples["tokens"][ind] = tl.split()      

    # print(examples['tokens'])

    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [24]:
tokenize_and_align_labels(datasets_ner['test'][:5])

{'input_ids': [[101, 152, 143, 158, 143, 156, 152, 163, 154, 167, 160, 147, 148, 151, 156, 147, 146, 168, 151, 156, 145, 151, 155, 158, 157, 160, 162, 161, 157, 148, 148, 125, 128, 119, 126, 158, 145, 162, 167, 160, 120, 167, 160, 119, 102], [101, 156, 143, 149, 143, 163, 160, 113, 122, 127, 125, 153, 151, 155, 151, 114, 102], [101, 145, 143, 160, 154, 151, 161, 154, 147, 117, 150, 163, 154, 154, 164, 144, 143, 160, 156, 147, 162, 117, 154, 147, 167, 162, 157, 156, 157, 160, 151, 147, 156, 162, 164, 150, 143, 160, 162, 154, 147, 158, 157, 157, 154, 117, 155, 143, 156, 161, 148, 151, 147, 154, 146, 164, 102], [101, 160, 147, 163, 162, 147, 160, 161, 150, 143, 161, 156, 157, 162, 164, 147, 160, 151, 148, 151, 147, 146, 162, 150, 147, 161, 147, 161, 162, 157, 160, 151, 147, 161, 143, 156, 146, 146, 157, 147, 161, 156, 157, 162, 164, 157, 163, 145, 150, 148, 157, 160, 162, 150, 147, 151, 160, 143, 145, 145, 163, 160, 143, 145, 167, 119, 102], [101, 158, 151, 147, 160, 149, 151, 163, 145, 1

In [25]:
tokenized_datasets = datasets_ner.map(tokenize_and_align_labels, batched=True)

# Training

In [26]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint a

In [27]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [28]:
args = TrainingArguments(
    f"test-ner-hashtag",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    save_total_limit = 1,
    save_steps = 500,
    eval_steps = 500,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [29]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [30]:
metric = load_metric("seqeval")


In [31]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [33]:
import wandb

!wandb login

#319ea6f7049796b3b25f922660a9f9595b7fc9c2


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [34]:
wandb.init(project="hashtag-seg-ner",name="run1-lower-cased-data-added", config={
    "learning_rate": 2e-5,
    "architecture": "mbert",
    "dataset": "random",
})
config = wandb.config

wandb: Currently logged in as: akshalabhatnagar (use `wandb login --relogin` to force relogin)


In [35]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags, idx.
***** Running training *****
  Num examples = 14088
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8805
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.539400,0.468515,0.618212,0.545090,0.579353,0.842109
1000,0.431800,0.399893,0.634848,0.622082,0.628400,0.856634
1500,0.389300,0.351923,0.781615,0.568021,0.657916,0.876448
2000,0.351800,0.326409,0.745607,0.642755,0.690371,0.886508
2500,0.316300,0.302683,0.773647,0.666904,0.716321,0.895361
3000,0.304400,0.287167,0.763995,0.705786,0.733738,0.898584
3500,0.290100,0.276307,0.772192,0.687578,0.727433,0.901375
4000,0.246800,0.269047,0.754478,0.745648,0.750037,0.904705
4500,0.236300,0.275650,0.727278,0.768669,0.747401,0.900868
5000,0.239300,0.265485,0.732094,0.788154,0.759090,0.905411


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags, idx.
***** Running Evaluation *****
  Num examples = 3522
  Batch size = 8
Saving model checkpoint to test-ner-hashtag/checkpoint-500
Configuration saved in test-ner-hashtag/checkpoint-500/config.json
Model weights saved in test-ner-hashtag/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner-hashtag/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner-hashtag/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags, idx.
***** Running Evaluation *****
  Num examples = 3522
  Batch size = 8
Saving model checkpoint to test-ner-hashtag/checkpoint-1000
Configuration saved in test-ner-hashtag/checkpoint-1000/config.json
Model weights saved in test-ner-hasht

TrainOutput(global_step=8805, training_loss=0.26542380103869984, metrics={'train_runtime': 3283.2428, 'train_samples_per_second': 21.454, 'train_steps_per_second': 2.682, 'total_flos': 1.039188258353016e+16, 'train_loss': 0.26542380103869984, 'epoch': 5.0})

In [36]:
trainer.save_model('/content/gdrive/MyDrive/hashtagseg/run1-lower-cased-data-added')

Saving model checkpoint to /content/gdrive/MyDrive/hashtagseg/run1-lower-cased-data-added
Configuration saved in /content/gdrive/MyDrive/hashtagseg/run1-lower-cased-data-added/config.json
Model weights saved in /content/gdrive/MyDrive/hashtagseg/run1-lower-cased-data-added/pytorch_model.bin
tokenizer config file saved in /content/gdrive/MyDrive/hashtagseg/run1-lower-cased-data-added/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/hashtagseg/run1-lower-cased-data-added/special_tokens_map.json


# Prediction

In [37]:
from transformers import AutoModel,AutoConfig,AutoModelForTokenClassification
config = AutoConfig.from_pretrained('/content/gdrive/MyDrive/hashtagseg/config.json')



model = AutoModelForTokenClassification.from_pretrained('/content/gdrive/MyDrive/hashtagseg',config=config)

loading configuration file /content/gdrive/MyDrive/hashtagseg/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",

In [38]:
model.to('cuda')

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [39]:
def predict_nertags(sentence):

  # print(sentence.split())

  # print(tokenizer.tokenize(list(sentence), is_split_into_words = True))

  # print(tokenizer(list(sentence), is_split_into_words=True, return_tensors='pt'))

  tokenized_sentence = tokenizer(list(sentence), is_split_into_words=True, return_tensors='pt')

  # print(tokenized_sentence)

  outputs = model(**tokenized_sentence.to('cuda'))

  # print(outputs.keys())

  preds = np.argmax(outputs['logits'].cpu().detach().numpy(), axis=2).squeeze()

  mask = []
  prev_id = None
  for ind,id in enumerate(tokenized_sentence.word_ids()):
    
    if id is None:
      mask.append(-100)
    elif id == prev_id:
      mask.append(-100)
    elif id != prev_id:
      mask.append(id)
    prev_id = id


  true_preds = [
      label_list[p] for (p, l) in zip(preds, mask) if l != -100
  ]

  # print(true_preds)

  print(sentence)
  print('*'*70)
  for word,pred in zip(list(sentence),true_preds,):
    print(word,'\t\t-',pred)

In [45]:
hashtag = 'ranilakshmibai'

predict_nertags(hashtag)

ranilakshmibai
**********************************************************************
r 		- O
a 		- O
n 		- O
i 		- O
l 		- O
a 		- O
k 		- O
s 		- O
h 		- O
m 		- O
i 		- O
b 		- O
a 		- O
i 		- O


In [46]:
hashtag = '#gazaunderattack'

predict_nertags(hashtag)

#gazaunderattack
**********************************************************************
# 		- O
g 		- O
a 		- O
z 		- O
a 		- O
u 		- B
n 		- O
d 		- O
e 		- O
r 		- O
a 		- O
t 		- O
t 		- O
a 		- O
c 		- O
k 		- O
